In [1]:
import pandas as pd
import numpy as np

Vamos a utilizar la regresion lineal de multiples dimensiones. Esto lo podemos hacer directamente ya que es equivalente a hacer una regresión lineal de 1 dimension pero utilizamos un vector para $x$ en lugar de un valor numérico. 

De hecho, al hacer las derivadas para el gradiente descendiente obtenemos que 

$\frac{∂J_{θ}}{∂θ_0} = \frac{1}{n}\sum_{i=1}^{n}(h_θ(x_i)-y_i)$

$\frac{∂J_{θ}}{∂θ_1} = \frac{1}{n}\sum_{i=1}^{n}(h_θ(x_i)-y_i)x_i \quad \forall \ i \neq 0$

que es justo las ecuaciones que utilizamos para otimizar los valores de $\theta$.

A continuacion implementaremos esta regresión multilineal de manera general a traves de una función para poder reutilizar el código más adelante. 

In [2]:
def multilinear_regression(x:np.array, y:np.array, initial:np.array, alpha:float=0.001, iters:int=1000) -> np.array:
    """This function train a multiple dimension linear model using gradient descent as its optimizer. 

    Args:
        x (np.array[np.array[float]]): The independent variables. Each row of variables must be an
        element of the array
        y (np.array[float]): An array with the dependant variable.
        initial (np.array[float]): The initial wieghts to consider.
        alpha (float): The learning rate
        iters (int): Number of iterations to run the model.

    Returns:
        np.array: The weights of the best fit. 
    """
    #For code simplicity, we'll add a 1-column for the constant factor
    x = np.array([np.insert(i, 0, 1) for i in x])

    #get the dimension
    dim = len(x[0])
    
    n = len(x)

    #define the hipothesis function
    h = lambda x, t: np.dot(x, t)

    t = initial
    for _ in range(iters):
        #computing dT
        t_delta = [1] * dim
        for i in range(dim):
            t_delta[i] = (1/n) * sum([(h(x[j], t) - y[j]) * x[j][i] for j in range(n)])
        
        t_delta = np.array(t_delta)
        #updating T
        t = (t - alpha*t_delta)
    
    return t

def predict_regression(t, x):
    x = np.array([np.insert(i, 0, 1) for i in x])
    return np.dot(x, t)


Vamos a probar esto con el dataset que se puede encontrar en `Data\salary.csv`. Estos son datos de profesores en una universidad con los cuales vamos a intentar predecir su salario. Como hay algunas variables categoricas vamos a hacer un one-hot encoding con algunas columnas. 

In [3]:
df = pd.read_csv('..\Data\salary.csv')
df = pd.get_dummies(df).drop(columns=['discipline_B', 'sex_Male'])
df

,yrs.since.phd,yrs.service,salary,rank_AssocProf,rank_AsstProf,rank_Prof,discipline_A,sex_Female
0,19,18,139750,False,False,True,False,False
1,20,16,173200,False,False,True,False,False
2,4,3,79750,False,True,False,False,False
3,45,39,115000,False,False,True,False,False
4,40,41,141500,False,False,True,False,False
...,...,...,...,...,...,...,...,...
392,33,30,103106,False,False,True,True,False
393,31,19,150564,False,False,True,True,False
394,42,25,101738,False,False,True,True,False
395,25,15,95329,False,False,True,True,False


Vamos a dividir nuestro dataset en un conjunto de pruebas otro de entrenamiento.

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns = 'salary'), df['salary'], test_size=0.2, random_state=42)

Ahora toca entrenar el modelo

In [5]:
t = multilinear_regression(X_train.values,
                           y_train.values,
                           np.array([10000, 50, 100, 50, 50, 50, 50, 50]),
                           0.0015,
                           10000)
t

array([ 74803.32630566,    917.18540936,   -404.32649677,  12446.35551641,
         9926.87427422,  42580.09651503, -14441.11793967,    613.6788751 ])

Generamos la predicción

In [6]:
y_pred = predict_regression(t, X_test.values)

Ahora vamos a calcular algunas metricas para medir el rendimiento del modelo

In [7]:
def get_metrics_regression(real, pred):
  mae = np.mean(np.abs(real - pred))
  mape = np.mean(np.abs(real - pred) / real)
  mse = np.mean((real - pred)**2)
  
  return mse, mae, mape
mse, mae, mape = get_metrics_regression(y_test, y_pred)
print(f'MSE:{mse}   MAE:{mae}   MAPE:{mape}')

MSE:613838312.0418055   MAE:17405.62856955903   MAPE:0.16573198127045846
